In [1]:
import pandas as pd

eunnho = pd.read_csv('./data/eunnho.csv')
jaehwan = pd.read_csv('./data/jaehwan.csv')
geunyoung = pd.read_csv('./data/geunyoung.csv')
hyeonyoung = pd.read_csv('./data/hyeonyoung.csv')

c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from pandas import DataFrame
def get_row_col_group(df: DataFrame):
  '''
  dfs['row0, col0'] -> row0, col0 dataframe
  '''
  grouped_row_col = df.groupby(['row', 'col'])
  dfs = {}
  for (row, col), group_df in grouped_row_col:
    name = f'row{row}, col{col}'
    dfs[name] = group_df[['x', 'y', 'x_rad', 'y_rad', 'z_rad', 'row', 'col', 'x_degree', 'y_degree', 'z_degree']]
    dfs[name] = dfs[name].reset_index().drop('index', axis=1)
  return dfs

eunnho_dfs = get_row_col_group(eunnho)
jaehwan_dfs = get_row_col_group(jaehwan)
geunyoung_dfs = get_row_col_group(geunyoung)
hyeonyoung_dfs = get_row_col_group(hyeonyoung)

In [3]:
from typing import Dict

def get_row_col_tupleArr(dfs: Dict[str, DataFrame]):
  '''
  points_dict['row0, col0'] -> [(1, 3), (2, 2), ...]
  '''
  points_dict = {}
  for name, grouped_df in dfs.items():
    points_dict[name] = list(zip(grouped_df['x'], grouped_df['y']))
  return points_dict

eunnho_points = get_row_col_tupleArr(eunnho_dfs)
jaehwan_points = get_row_col_tupleArr(jaehwan_dfs)
geunyoung_points = get_row_col_tupleArr(geunyoung_dfs)
hyeonyoung_points = get_row_col_tupleArr(hyeonyoung_dfs)

In [18]:
import smallestenclosingcircle
def get_smallest_closing_circle(points_dict: Dict):
  circle_dict = {}
  for name, points in points_dict.items():
    center_x, center_y, radius = smallestenclosingcircle.make_circle(points)
    circle_dict[name] = {
    'x': center_x,
    'y': center_y,
    'r': radius
    }
  return circle_dict

eunnho_circle = get_smallest_closing_circle(eunnho_points)
jaehwan_circle = get_smallest_closing_circle(points_dict=jaehwan_points)
geunyoung_circle = get_smallest_closing_circle(geunyoung_points)
hyeonyoung_circle = get_smallest_closing_circle(hyeonyoung_points)

In [84]:
from sklearn.cluster import KMeans
import numpy as np
def get_cluster_center(dfs: Dict[str, DataFrame]):
  center_dict = {}
  for name, group_df in dfs.items():
    x = group_df['x']
    y = group_df['y']
    data = np.array(list(zip(x, y)))
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(data)
    center_x, center_y = kmeans.cluster_centers_[0]
    center_dict[name] = {'x': round(center_x), 'y': round(center_y)}
  return center_dict

eunnho_center = get_cluster_center(eunnho_dfs)
jaehwan_center = get_cluster_center(jaehwan_dfs)
geunyoung_center = get_cluster_center(geunyoung_dfs)
hyeonyoung_center = get_cluster_center(hyeonyoung_dfs)

c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\KHU\anaconda3\envs\py3_8_5\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will chang

In [90]:
eunnho_center['row1, col1']

{'x': 208, 'y': 386}

In [80]:
eunnho_circle['row1, col1']

{'x': 222.5, 'y': 376.5, 'r': 36.50342449688796}

In [92]:
rec_center_x = [83, 195, 307]
rec_center_y = [183, 375, 567]

import math
def get_angle(recx, recy, circlex, circley):
  '''
  width: |rcosθ|, height: |rsinθ| 
  1) 0 <= θ < 90: 원이 1사분면, x: →, y ↑
  2) 90 <= θ < 180: 원이 2사분면, x: ←, y ↑
  3) -180 <= θ < -90: 원이 3사분면, x: ←, y ↓
  4) -90 <= θ < 0: 원이 4 사분면, x: →, y ↓
  '''
  angle_rad = math.atan2(circley - recy, circlex - recx)
  angle_deg = math.degrees(angle_rad)
  if angle_deg >= 0 and angle_deg < 90:
    return angle_rad, 4
  elif angle_deg >= 90 and angle_deg < 180:
    return angle_rad, 3
  elif angle_deg >= -180 and angle_deg < -90:
    return angle_rad, 2
  else:
    return angle_rad, 1
  
def get_increase(r, rad):
  width = round(abs(r * math.cos(rad)))
  height = round(abs(r * math.sin(rad)))
  return width, height

def getwh(recx, recy, circlex, circley):
  radius = math.dist([recx, recy], [circlex, circley])
  rad, pos = get_angle(recx, recy, circlex, circley)
  w, h = get_increase(radius, rad)
  return {'width': w, 'height': h, 'pos': pos}



In [93]:
eunnho_wh = {
  '0': getwh(83, 183, eunnho_center['row0, col0']['x'], eunnho_center['row0, col0']['y']),
  '1': getwh(195, 183, eunnho_center['row0, col1']['x'], eunnho_center['row0, col1']['y']),
  '2': getwh(307, 183, eunnho_center['row0, col2']['x'], eunnho_center['row0, col2']['y'] ),
  '3': getwh(83, 375, eunnho_center['row1, col0']['x'], eunnho_center['row1, col0']['y'] ),
  '4': getwh(195, 375, eunnho_center['row1, col1']['x'], eunnho_center['row1, col1']['y'] ),
  '5': getwh(307, 375, eunnho_center['row1, col2']['x'], eunnho_center['row1, col2']['y'] ),
  '6': getwh(83, 567, eunnho_center['row2, col0']['x'], eunnho_center['row2, col0']['y'] ),
  '7': getwh(195, 567, eunnho_center['row2, col1']['x'], eunnho_center['row2, col1']['y'] ),
  '8': getwh(307, 567, eunnho_center['row2, col2']['x'], eunnho_center['row2, col2']['y'] ),
}

jaehwan_wh = {
  '0': getwh(83, 183, jaehwan_center['row0, col0']['x'], jaehwan_center['row0, col0']['y']),
  '1': getwh(195, 183, jaehwan_center['row0, col1']['x'], jaehwan_center['row0, col1']['y']),
  '2': getwh(307, 183, jaehwan_center['row0, col2']['x'], jaehwan_center['row0, col2']['y']),
  '3': getwh(83, 375, jaehwan_center['row1, col0']['x'], jaehwan_center['row1, col0']['y']),
  '4': getwh(195, 375, jaehwan_center['row1, col1']['x'], jaehwan_center['row1, col1']['y']),
  '5': getwh(307, 375, jaehwan_center['row1, col2']['x'], jaehwan_center['row1, col2']['y']),
  '6': getwh(83, 567, jaehwan_center['row2, col0']['x'], jaehwan_center['row2, col0']['y']),
  '7': getwh(195, 567, jaehwan_center['row2, col1']['x'], jaehwan_center['row2, col1']['y']),
  '8': getwh(307, 567, jaehwan_center['row2, col2']['x'], jaehwan_center['row2, col2']['y']),
}

geunyoung_wh = {
  '0': getwh(83, 183, geunyoung_center['row0, col0']['x'], geunyoung_center['row0, col0']['y']),
  '1': getwh(195, 183, geunyoung_center['row0, col1']['x'], geunyoung_center['row0, col1']['y']),
  '2': getwh(307, 183, geunyoung_center['row0, col2']['x'], geunyoung_center['row0, col2']['y']),
  '3': getwh(83, 375, geunyoung_center['row1, col0']['x'], geunyoung_center['row1, col0']['y']),
  '4': getwh(195, 375, geunyoung_center['row1, col1']['x'], geunyoung_center['row1, col1']['y']),
  '5': getwh(307, 375, geunyoung_center['row1, col2']['x'], geunyoung_center['row1, col2']['y']),
  '6': getwh(83, 567, geunyoung_center['row2, col0']['x'], geunyoung_center['row2, col0']['y']),
  '7': getwh(195, 567, geunyoung_center['row2, col1']['x'], geunyoung_center['row2, col1']['y']),
  '8': getwh(307, 567, geunyoung_center['row2, col2']['x'], geunyoung_center['row2, col2']['y']),
}

hyeonyoung_wh = {
  '0': getwh(83, 183, hyeonyoung_center['row0, col0']['x'], hyeonyoung_center['row0, col0']['y']),
  '1': getwh(195, 183, hyeonyoung_center['row0, col1']['x'], hyeonyoung_center['row0, col1']['y']),
  '2': getwh(307, 183, hyeonyoung_center['row0, col2']['x'], hyeonyoung_center['row0, col2']['y']),
  '3': getwh(83, 375, hyeonyoung_center['row1, col0']['x'], hyeonyoung_center['row1, col0']['y']),
  '4': getwh(195, 375, hyeonyoung_center['row1, col1']['x'], hyeonyoung_center['row1, col1']['y']),
  '5': getwh(307, 375, hyeonyoung_center['row1, col2']['x'], hyeonyoung_center['row1, col2']['y']),
  '6': getwh(83, 567, hyeonyoung_center['row2, col0']['x'], hyeonyoung_center['row2, col0']['y']),
  '7': getwh(195, 567, hyeonyoung_center['row2, col1']['x'], hyeonyoung_center['row2, col1']['y']),
  '8': getwh(307, 567, hyeonyoung_center['row2, col2']['x'], hyeonyoung_center['row2, col2']['y']),
}



{'0': {'width': 27, 'height': 22, 'pos': 4},
 '1': {'width': 20, 'height': 22, 'pos': 4},
 '2': {'width': 4, 'height': 28, 'pos': 4},
 '3': {'width': 24, 'height': 9, 'pos': 4},
 '4': {'width': 13, 'height': 11, 'pos': 4},
 '5': {'width': 7, 'height': 12, 'pos': 4},
 '6': {'width': 5, 'height': 20, 'pos': 1},
 '7': {'width': 9, 'height': 9, 'pos': 1},
 '8': {'width': 9, 'height': 2, 'pos': 4}}

In [96]:
eunnho_wh, jaehwan_wh, geunyoung_wh, hyeonyoung_wh

({'0': {'width': 27, 'height': 22, 'pos': 4},
  '1': {'width': 20, 'height': 22, 'pos': 4},
  '2': {'width': 4, 'height': 28, 'pos': 4},
  '3': {'width': 24, 'height': 9, 'pos': 4},
  '4': {'width': 13, 'height': 11, 'pos': 4},
  '5': {'width': 7, 'height': 12, 'pos': 4},
  '6': {'width': 5, 'height': 20, 'pos': 1},
  '7': {'width': 9, 'height': 9, 'pos': 1},
  '8': {'width': 9, 'height': 2, 'pos': 4}},
 {'0': {'width': 18, 'height': 17, 'pos': 4},
  '1': {'width': 19, 'height': 21, 'pos': 4},
  '2': {'width': 4, 'height': 25, 'pos': 4},
  '3': {'width': 19, 'height': 11, 'pos': 4},
  '4': {'width': 12, 'height': 9, 'pos': 4},
  '5': {'width': 4, 'height': 3, 'pos': 4},
  '6': {'width': 7, 'height': 25, 'pos': 1},
  '7': {'width': 12, 'height': 17, 'pos': 1},
  '8': {'width': 2, 'height': 9, 'pos': 1}},
 {'0': {'width': 24, 'height': 20, 'pos': 4},
  '1': {'width': 20, 'height': 22, 'pos': 4},
  '2': {'width': 5, 'height': 27, 'pos': 4},
  '3': {'width': 22, 'height': 7, 'pos': 4},
  '

In [94]:
import pickle

with open('./data/whdata/eunnho.pickle', 'wb') as fw:
  pickle.dump(eunnho_wh, fw)

with open('./data/whdata/jaehwan.pickle', 'wb') as fw:
  pickle.dump(jaehwan_wh, fw)

with open('./data/whdata/geunyoung.pickle', 'wb') as fw:
  pickle.dump(geunyoung_wh, fw)

with open('./data/whdata/hyeonyoung.pickle', 'wb') as fw:
  pickle.dump(hyeonyoung_wh, fw)

In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb

eunnho_df = pd.read_csv('./data/eunnho.csv')
jaehwan_df = pd.read_csv('./data/jaehwan.csv')
geunyoung_df = pd.read_csv('./data/geunyoung.csv')
hyeonyoung_df = pd.read_csv('./data/hyeonyoung.csv')

In [63]:
X = hyeonyoung_df[['acc_x', 'acc_y', 'acc_z']].values
y = hyeonyoung_df['idx'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
d_train = lgb.Dataset(X_train, label=y_train)

In [64]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_score
params = {
    'objective': 'multiclass',
    'num_class': 9,  # 클래스의 수 (0부터 8까지 9개의 클래스)
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

clf = lgb.train(params, d_train, 1000)
y_pred_1=clf.predict(X_test)

y_pred_2 = [np.argmax(line) for line in y_pred_1]
precision_score(y_pred_2, y_test, average=None).mean()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 387
[LightGBM] [Info] Number of data points in the train set: 384, number of used features: 3
[LightGBM] [Info] Start training from score -2.189442
[LightGBM] [Info] Start training from score -2.261763
[LightGBM] [Info] Start training from score -2.100495
[LightGBM] [Info] Start training from score -2.189442
[LightGBM] [Info] Start training from score -2.166453
[LightGBM] [Info] Start training from score -2.189442
[LightGBM] [Info] Start training from score -2.212973
[LightGBM] [Info] Start training from score -2.166453
[LightGBM] [Info] Start training from score -2.313056
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

0.6154524487857821

In [106]:
np.round(y_pred_1, decimals=2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.98, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.01, 0.59, 0.38, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.59, 0.  , 0.41, 0.  ],
       [0.  , 0.94, 0.06, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.95, 0.  , 0.  , 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.26, 0.  , 0.74, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.82, 0.  , 0.12,

In [67]:
X_test

array([[  0.70853405,  -6.39162061,  -7.99798028],
       [  4.55960801,  -8.5182714 ,  -3.54386795],
       [  3.18099589,  -6.13813392,  -7.57315913],
       [  2.70978707,  -8.7373391 ,  -3.25790962],
       [  3.68452672,  -7.95742849,  -4.20182391],
       [  0.71167637,  -5.56891306,  -9.62289502],
       [  2.92930602,  -7.84355428,  -6.30857174],
       [  3.253571  ,  -8.87306216,  -3.41039063],
       [  3.0352493 ,  -6.04446137,  -5.17042871],
       [  3.2996588 ,  -5.59270544,  -8.6013191 ],
       [ -0.52447989,  -5.36121548,  -9.10200652],
       [  0.43933593,  -5.20125347,  -9.560796  ],
       [  4.04395595,  -8.10332457,  -6.1894602 ],
       [  1.71754042,  -5.87716652,  -9.81996775],
       [  0.42018239,  -7.05676034,  -9.37823812],
       [  0.57400974,  -5.62966534,  -9.29099882],
       [  3.36175841,  -5.52506924,  -8.24009393],
       [  0.3465606 ,  -4.21349565, -10.38200659],
       [  3.81501042,  -8.4747251 ,  -5.59869065],
       [  1.38055682,  -7.30261

In [107]:
res = np.round(clf.predict([[-0.188543, -4.746805, -8.906280]]), decimals=2)

In [108]:
result_dict = {index: value for index, value in enumerate(res[0])}
result_dict

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 1.0}

In [98]:
with open('./data/whdata/eunnho.pickle', 'rb') as f:
  wh = pickle.load(f)
wh

{'0': {'width': 27, 'height': 22, 'pos': 4},
 '1': {'width': 20, 'height': 22, 'pos': 4},
 '2': {'width': 4, 'height': 28, 'pos': 4},
 '3': {'width': 24, 'height': 9, 'pos': 4},
 '4': {'width': 13, 'height': 11, 'pos': 4},
 '5': {'width': 7, 'height': 12, 'pos': 4},
 '6': {'width': 5, 'height': 20, 'pos': 1},
 '7': {'width': 9, 'height': 9, 'pos': 1},
 '8': {'width': 9, 'height': 2, 'pos': 4}}

In [103]:
with open('./data/whdata/eunnho.pickle', 'rb') as f:
  wh = pickle.load(f)

for key, value in wh.items():
  value['height'] = value['height'] * result_dict[int(key)]
  value['width'] = value['width'] * result_dict[int(key)]

In [104]:
wh

{'0': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '1': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '2': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '3': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '4': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '5': {'width': 0.0, 'height': 0.0, 'pos': 4},
 '6': {'width': 0.0, 'height': 0.0, 'pos': 1},
 '7': {'width': 0.0, 'height': 0.0, 'pos': 1},
 '8': {'width': 9.0, 'height': 2.0, 'pos': 4}}

In [65]:
import joblib
joblib.dump(clf, './model/hyeonyoung.pkl')

['./model/hyeonyoung.pkl']